### Introduccion
Implementar un modelo de ML para clasificar si un dominio es legítimo o es Phishing. El phising se basa en la ingeniería social (manipulación de emociones, aprovechamiento de atajos mentales y sesgos cognitivos) para engañar a las víctimas y lograr que estas den información (normalmente credenciales).

### Parte I --- Ingeniería de características

### Explotación de datos

1. Cargue el dataset en un dataframe de pandas, muestre un ejemplo de cinco observaciones.

In [4]:
import pandas as pd

data = pd.read_csv('dataset_phishing.csv')
data.head(5)

,url,status
0,http://www.crestonwood.com/router.php,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,phishing
2,https://support-appleld.com.secureupdate.duila...,phishing
3,http://rgipt.ac.in,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,legitimate
